In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Set the path to the dataset
dataset_path = r'D:\IIT\Subjects\(4605)IRP\Devlo\DataSet'

In [3]:
# Define image dimensions and other constants
img_height, img_width = 256, 256  # Image resolution
batch_size = 32
num_classes = 4  # 4 classes: Healthy, Live Wood, Pink Wax, Stem Canker

In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 80% training, 20% validation
)

In [5]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training subset
)

Found 1554 images belonging to 4 classes.


In [6]:
# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation subset
)

Found 388 images belonging to 4 classes.


In [7]:
# Build CNN model
model = Sequential()

In [8]:
# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [9]:
# Flatten and Fully Connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with 4 classes

In [10]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Adjust the number of epochs as needed
)

Epoch 1/10
48/48 [==============================] - 129s 3s/step - loss: 1.4285 - accuracy: 0.3509 - val_loss: 1.0130 - val_accuracy: 0.5026
Epoch 2/10
48/48 [==============================] - 103s 2s/step - loss: 1.1477 - accuracy: 0.5059 - val_loss: 0.8779 - val_accuracy: 0.6094
Epoch 3/10
48/48 [==============================] - 109s 2s/step - loss: 1.0199 - accuracy: 0.5848 - val_loss: 0.7831 - val_accuracy: 0.6745
Epoch 4/10
48/48 [==============================] - 109s 2s/step - loss: 0.9211 - accuracy: 0.6229 - val_loss: 0.6867 - val_accuracy: 0.6927
Epoch 5/10
48/48 [==============================] - 108s 2s/step - loss: 0.8352 - accuracy: 0.6564 - val_loss: 0.6890 - val_accuracy: 0.6641
Epoch 6/10
48/48 [==============================] - 108s 2s/step - loss: 0.7413 - accuracy: 0.7135 - val_loss: 0.6429 - val_accuracy: 0.6849
Epoch 7/10
48/48 [==============================] - 109s 2s/step - loss: 0.6506 - accuracy: 0.7451 - val_loss: 0.4237 - val_accuracy: 0.8568
Epoch 8/10
48

In [13]:
# Save the model
model.save('tea_stem_disease_model_4_classes.h5')

C:\Users\ridmi\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# Evaluate the model
score = model.evaluate(validation_generator)
print(f"Validation Loss: {score[0]}")
print(f"Validation Accuracy: {score[1]}")

13/13 [==============================] - 10s 769ms/step - loss: 0.4982 - accuracy: 0.7758
Validation Loss: 0.49818482995033264
Validation Accuracy: 0.7757731676101685
